<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

# Radiant Earth Spot the Crop Challenge [Sentinel]

## Radiant MLHub API


The Radiant MLHub API gives access to open Earth imagery training data for machine learning applications. You can learn more about the repository at the [Radiant MLHub site](https://mlhub.earth) and about the organization behind it at the [Radiant Earth Foundation site](https://radiant.earth).

Full documentation for the API is available at [docs.mlhub.earth](docs.mlhub.earth).

Each item in our collection is explained in json format compliant with [STAC](https://stacspec.org/) [label extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/label) definition.

## Dependencies

All the dependencies for this notebook are included in the `requirements.txt` file included in this folder.


**You must replace the `YOUR_API_KEY_HERE` text with your API key which you can obtain by creating a free account on the [MLHub Dashboard](https://dashboard.mlhub.earth/) within the `API Keys` tab at the top of the page.**

In [2]:
!pip install radiant-mlhub
!pip install rasterio

In [3]:
# Required libraries
import os
import tarfile
import json
from pathlib import Path
from radiant_mlhub.client import _download as download_file

import datetime
import rasterio
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit

os.environ['MLHUB_API_KEY'] = '315862edef3b93d071bf6729326dc682929c0e6d537a3e1da51274b5a8cd0a13'

In [4]:
DOWNLOAD_S1 = False # If you set this to true then the Sentinel-1 data will be downloaded which is not needed in this notebook.

# Select which imagery bands you'd like to download here:
DOWNLOAD_BANDS = {
    'B01': False,
    'B02': False,
    'B03': True,
    'B04': True,
    'B05': False,
    'B06': False,
    'B07': False,
    'B08': True,
    'B8A': False,
    'B09': False,
    'B11': False,
    'B12': False,
    'CLM': True
}

# In this model we will only use Green, Red and NIR bands. You can select to download any number of bands. 
# Our choice relies on the fact that vegetation is most sensitive to these bands. 
# We also donwload the CLM or Cloud Mask layer to exclude cloudy data from the training phase. 
# You can also do a feature selection, and try different combination of bands to see which ones will result in a better accuracy. 

## Downloading and Loading the Data

In this part, we will download the data from Radiant MLHub and load the properties of each item in the dataset into a DataFrame


In [5]:
FOLDER_BASE = 'ref_south_africa_crops_competition_v1'

def download_archive(archive_name):
    if os.path.exists(archive_name.replace('.tar.gz', '')):
        return
    
    print(f'Downloading {archive_name} ...')
    download_url = f'https://radiant-mlhub.s3.us-west-2.amazonaws.com/archives/{archive_name}'
    download_file(download_url, '.')
    print(f'Extracting {archive_name} ...')
    with tarfile.open(archive_name) as tfile:
        tfile.extractall()
    os.remove(archive_name)

for split in ['train', 'test']:
    # Download the labels
    labels_archive = f'{FOLDER_BASE}_{split}_labels.tar.gz'
    download_archive(labels_archive)
    
    # Download Sentinel-1 data
    if DOWNLOAD_S1:
        s1_archive = f'{FOLDER_BASE}_{split}_source_s1.tar.gz'
        download_archive(s1_archive)
        

    for band, download in DOWNLOAD_BANDS.items():
        if not download:
            continue
        s2_archive = f'{FOLDER_BASE}_{split}_source_s2_{band}.tar.gz'
        download_archive(s2_archive)
        
def resolve_path(base, path):
    return Path(os.path.join(base, path)).resolve()
        
def load_df(collection_id):
    split = collection_id.split('_')[-2]
    collection = json.load(open(f'{collection_id}/collection.json', 'r'))
    rows = []
    item_links = []
    for link in collection['links']:
        if link['rel'] != 'item':
            continue
        item_links.append(link['href'])
        
    for item_link in item_links:
        item_path = f'{collection_id}/{item_link}'
        current_path = os.path.dirname(item_path)
        item = json.load(open(item_path, 'r'))
        tile_id = item['id'].split('_')[-1]
        for asset_key, asset in item['assets'].items():
            rows.append([
                tile_id,
                None,
                None,
                asset_key,
                str(resolve_path(current_path, asset['href']))
            ])
            
        for link in item['links']:
            if link['rel'] != 'source':
                continue
            source_item_id = link['href'].split('/')[-2]
            
            if source_item_id.find('_s1_') > 0 and not DOWNLOAD_S1:
                continue
            elif source_item_id.find('_s1_') > 0:
                for band in ['VV', 'VH']:
                    asset_path = Path(f'{FOLDER_BASE}_{split}_source_s1/{source_item_id}/{band}.tif').resolve()
                    date = '-'.join(source_item_id.split('_')[10:13])
                    
                    rows.append([
                        tile_id,
                        f'{date}T00:00:00Z',
                        's1',
                        band,
                        asset_path
                    ])
                
            if source_item_id.find('_s2_') > 0:
                for band, download in DOWNLOAD_BANDS.items():
                    if not download:
                        continue
                    
                    asset_path = Path(f'{FOLDER_BASE}_{split}_source_s2_{band}/{source_item_id}_{band}.tif').resolve()
                    date = '-'.join(source_item_id.split('_')[10:13])
                    rows.append([
                        tile_id,
                        f'{date}T00:00:00Z',
                        's2',
                        band,
                        asset_path
                    ])
            
    return pd.DataFrame(rows, columns=['tile_id', 'datetime', 'satellite_platform', 'asset', 'file_path'])

competition_train_df = load_df(f'{FOLDER_BASE}_train_labels')
competition_test_df = load_df(f'{FOLDER_BASE}_test_labels')

In [ ]:
competition_train_df

,tile_id,datetime,satellite_platform,asset,file_path
0,2587,None,None,documentation,/content/ref_south_africa_crops_competition_v1...
1,2587,None,None,field_ids,/content/ref_south_africa_crops_competition_v1...
2,2587,None,None,field_info_train,/content/ref_south_africa_crops_competition_v1...
3,2587,None,None,labels,/content/ref_south_africa_crops_competition_v1...
4,2587,None,None,raster_values,/content/ref_south_africa_crops_competition_v1...
...,...,...,...,...,...
591109,2198,2017-11-27T00:00:00Z,s2,CLM,/content/ref_south_africa_crops_competition_v1...
591110,2198,2017-11-30T00:00:00Z,s2,B03,/content/ref_south_africa_crops_competition_v1...
591111,2198,2017-11-30T00:00:00Z,s2,B04,/content/ref_south_africa_crops_competition_v1...
591112,2198,2017-11-30T00:00:00Z,s2,B08,/content/ref_south_africa_crops_competition_v1...


In [ ]:
competition_test_df

,tile_id,datetime,satellite_platform,asset,file_path
0,0590,None,None,documentation,/content/ref_south_africa_crops_competition_v1...
1,0590,None,None,field_ids,/content/ref_south_africa_crops_competition_v1...
2,0590,None,None,field_info_test,/content/ref_south_africa_crops_competition_v1...
3,0590,None,None,raster_values,/content/ref_south_africa_crops_competition_v1...
4,0590,None,None,sample_submission,/content/ref_south_africa_crops_competition_v1...
...,...,...,...,...,...
252580,0947,2017-11-27T00:00:00Z,s2,CLM,/content/ref_south_africa_crops_competition_v1...
252581,0947,2017-11-30T00:00:00Z,s2,B03,/content/ref_south_africa_crops_competition_v1...
252582,0947,2017-11-30T00:00:00Z,s2,B04,/content/ref_south_africa_crops_competition_v1...
252583,0947,2017-11-30T00:00:00Z,s2,B08,/content/ref_south_africa_crops_competition_v1...


In [ ]:
# This DataFrame lists all types of assets including documentation of the data. 
# In the following, we will use the Sentinel-2 bands as well as labels. 
competition_train_df['asset'].unique()

array(['documentation', 'field_ids', 'field_info_train', 'labels',
       'raster_values', 'B03', 'B04', 'B08', 'CLM'], dtype=object)

In [6]:
tile_ids_train = competition_train_df['tile_id'].unique()

In [7]:
# We are selecting the n_obs cloud free images. Ideally, you should
# select the images across the different tiles with the same temporal frequency. 
n_obs = 3

In [ ]:
np.count_nonzero(~np.isnan(field_ids))

173670400

In [ ]:
X.shape

(256, 256, 10662)

In [ ]:
# Our goal is developing a pixel-based Random Forest model. So we will create an X variable
# that each row is a pixel and each column is one of the observations. 
# The other variables is y which has rows equal to the number of pixels. 
X = np.empty((256, 256, 0), dtype=np.float16)
X = []

for i in range (0,len(tile_ids_train)):
    if i%200 == 0:
        print(i)
    tile_id = tile_ids_train[i]  
  
    
    tile_df = competition_train_df[competition_train_df['tile_id']==tile_id]

    tile_date_times = tile_df[tile_df['satellite_platform']=='s2']['datetime'].unique()

    X_tile = np.empty((256, 256, 0), dtype=np.uint8)
    n_X = 0
    for date_time in tile_date_times:
        # Here we retrieve the cloud band, and check if it's cloud free we will load the other bands
        # Otherwise we will pass on to the next observation
        
        clm_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='CLM')]['file_path'].values[0])
        clm_max = np.max(clm_src.read(1))
        # In the following we select images that the maximum cloud cover probability per pixel is 10% (10% * 255 = 25.5).
        if clm_max < 25:
            n_X+=1
            b3_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B03')]['file_path'].values[0])
            b3_array = np.expand_dims(b3_src.read(1), axis=2)

            b4_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B04')]['file_path'].values[0])
            b4_array = np.expand_dims(b4_src.read(1), axis=2)

            b8_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B08')]['file_path'].values[0])
            b8_array = np.expand_dims(b8_src.read(1), axis=2)

            #print(b3_array.shape)

            X_tile = np.empty((256, 256, 0), dtype=np.uint8)
            X_tile = np.append(X_tile, b3_array, axis = 2)
            X_tile = np.append(X_tile, b4_array, axis = 2)
            X_tile = np.append(X_tile, b8_array, axis = 2)
            X.append(X_tile)
        if n_X == n_obs:
            break
X = np.array(X)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600


In [ ]:
X = np.true_divide(X, 255.0)
print(type(X[0][0][0][0]))
X = np.array(X, dtype=np.float16)
type(X[0][0][0][0])

<class 'numpy.float64'>


numpy.float16

In [ ]:
#y_t = np.empty((256, 256,0), dtype=np.uint8)
y_t = []
ll = [0,1,2,3,4,5,6,7,8,9]
for i in range (0,len(tile_ids_train)):
    if i%500 == 0:
        print(i)
    tile_id = tile_ids_train[i]  
    
    tile_df = competition_train_df[competition_train_df['tile_id']==tile_id]

    label_src = rasterio.open(tile_df[tile_df['asset']=='labels']['file_path'].values[0])
    label_array = label_src.read(1)
    y_3 = np.empty((256, 256, 0), dtype=np.uint8)
    y__l = np.expand_dims(label_array, axis=2)
    for i_l in ll:
        y_3 = np.append(y_3, (y__l == i_l).astype(np.uint8), axis = 2)
    for y_t_i in range(0,n_obs):
        y_t.append(y_3)
y_t = np.array(y_t,dtype=np.float32)
print(np.count_nonzero(~np.isnan(y_t)))
y_t.shape

0
500
1000
1500
2000
2500
1736704000


(2650, 256, 256, 10)

In [ ]:
np.unique(np.around(y_t[3][:][:][1]), return_counts=True)

(array([0, 1], dtype=uint8), array([2304,  256]))

In [ ]:
X

array([[43., 56., 75., ..., 26., 35., 61.],
       [33., 51., 68., ..., 24., 30., 60.],
       [31., 43., 62., ..., 21., 26., 54.],
       ...,
       [24., 36., 48., ..., 17., 30., 46.],
       [21., 29., 41., ..., 14., 21., 36.],
       [21., 29., 41., ..., 16., 27., 39.]])

## Building the Model

In [ ]:
# Each field has several pixels in the data. Here our goal is to build a Random Forest (RF) model using the average values
# of the pixels within each field. So, we use `groupby` to take the mean for each field_id
data_grouped = data.groupby('field_id').mean().reset_index()
data_grouped

,field_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,label
0,0.0,34.500000,50.900000,76.800000,33.000000,49.300000,73.600000,33.700000,50.200000,73.700000,31.900000,46.600000,69.800000,28.800000,41.600000,65.600000,4.9
1,2.0,32.043478,46.913043,73.913043,34.391304,46.695652,73.347826,24.000000,36.173913,56.478261,26.739130,37.695652,55.391304,20.782609,27.565217,49.347826,7.0
2,4.0,45.847682,61.059603,82.986755,40.158940,54.165563,76.940397,28.092715,39.218543,58.317881,35.629139,47.847682,67.000000,45.384106,59.947020,81.132450,8.0
3,19.0,22.266160,31.463878,85.053232,18.950570,28.646388,75.806084,18.699620,29.395437,74.882129,17.980989,29.159696,66.220532,25.114068,32.904943,62.574144,4.0
4,29.0,28.278689,40.229508,68.836066,23.885246,34.213115,56.737705,25.065574,34.868852,55.147541,24.409836,33.065574,50.327869,17.540984,24.344262,41.803279,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16317,122715.0,11.259259,9.851852,66.962963,10.888889,10.333333,66.851852,19.962963,27.814815,58.259259,25.185185,35.962963,63.962963,18.407407,27.555556,47.037037,3.0
16318,122716.0,44.683333,58.916667,70.216667,39.366667,51.816667,59.983333,40.200000,54.666667,65.900000,36.133333,48.000000,57.150000,35.533333,47.050000,55.900000,6.0
16319,122723.0,20.611429,30.177143,54.902857,21.257143,30.674286,55.708571,20.154286,29.011429,54.468571,17.411429,23.942857,44.234286,26.594286,37.474286,58.320000,3.0
16320,122725.0,41.490854,58.823171,79.570122,37.679878,54.625000,76.216463,41.143293,59.006098,80.762195,39.548780,57.457317,79.734756,36.865854,53.128049,71.128049,5.0


In [ ]:
# Split train and test
# We use field_ids to split the data to train and test. Note that the test portion for training is different than the test 
# portion provided as part of the competition. 
train_per = 0.7

n_fields = len(data_grouped['field_id'])
np.random.seed(10)
train_fields = np.random.choice(data_grouped['field_id'], int(n_fields * train_per), replace=False)
test_fields = data_grouped['field_id'][~np.in1d(data_grouped['field_id'], train_fields)]

In [ ]:
X_train, X_test = data_grouped[data_grouped['field_id'].isin(train_fields)], data_grouped[data_grouped['field_id'].isin(test_fields)]
X_train = X_train.drop(columns=['label', 'field_id'])
X_test = X_test.drop(columns=['label', 'field_id'])
y_train, y_test = data_grouped[data_grouped['field_id'].isin(train_fields)]['label'], data_grouped[data_grouped['field_id'].isin(test_fields)]['label']

#### RF model

In [ ]:
# We ran a simple hyperparameter tuning for the number of trees, and concluded to use:
n_trees = 100

In [ ]:
# Fitting the RF model
rf = RandomForestClassifier(n_estimators = n_trees, random_state = 0, n_jobs = 3, verbose =1)
rf.fit(X_train, y_train.astype(int))

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.8s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    1.6s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=3,
                       oob_score=False, random_state=0, verbose=1,
                       warm_start=False)

**NN Model**

In [6]:
!pip install segmentation_models
import segmentation_models as sm

sm.set_framework('tf.keras')

sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [ ]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y_t, test_size=0.4, random_state=42)

In [ ]:
del y_t
import gc

In [ ]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train[:1000], y_train[:1000]))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_val[:1000], y_val[:1000]))

In [ ]:
train_tf_dataset.as_numpy_iterator()
for sales in train_tf_dataset.as_numpy_iterator():
    print(sales, len(sales))
    del sales
    break

2


In [ ]:
BATCH_SIZE = 32
train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [ ]:
train_dataset

<PrefetchDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 10)), types: (tf.float16, tf.float32)>

In [ ]:
y_train = np.array(y_train, dtype=np.float32)

In [ ]:
y_val = np.array(y_val, dtype=np.float32)

In [ ]:
#del X_train
#del X_val
gc.collect()

23171

In [ ]:
type(X_train[0][0][0][0])

numpy.float16

In [ ]:
model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

In [ ]:
model.trainable = False
for layer in model.layers[-20:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

In [ ]:
import segmentation_models as sm
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# preprocess input
x_train = preprocess_input(X_train[:1200])
x_val = preprocess_input(X_val[:1000])
y_train = y_train[:1200]
y_val = y_val[:1000]

# define model
# multiclass segmentation with non overlapping class masks (your classes + background)
model = sm.Unet('resnet34', encoder_weights='imagenet', input_shape=(256, 256, 3), classes= 10, decoder_use_batchnorm=True, activation='softmax', encoder_freeze = True)

from tensorflow.keras.models import Model
model= Model(inputs=model.input, outputs=model.layers[-1].output)

from tensorflow.keras import layers
for layer in model.layers[:]:
    print(layer.trainable, layer)

model.layers[3].trainable = True

callbacks = [
        ModelCheckpoint("model.h5", verbose=1, save_best_model=True),
        ReduceLROnPlateau(monitor="val_loss", patience=10, factor=0.1, verbose=1, min_lr=1e-5),
        #EarlyStopping(monitor="val_loss", patience=5, verbose=1)
    ]
batch_size=8
train_steps = len(x_train)//batch_size
valid_steps = len(x_val)//batch_size
    
model.compile(
    
    tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=sm.losses.categorical_focal_jaccard_loss,
    metrics=[sm.metrics.iou_score, 'mse'],
    
)

model.summary()


False <keras.engine.input_layer.InputLayer object at 0x7f440c350f90>
True <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f440c350450>
False <keras.layers.convolutional.ZeroPadding2D object at 0x7f4304d22750>
False <keras.layers.convolutional.Conv2D object at 0x7f440c2a8d10>
True <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f440c30fe50>
False <keras.layers.core.Activation object at 0x7f440c315910>
False <keras.layers.convolutional.ZeroPadding2D object at 0x7f440c322150>
False <keras.layers.pooling.MaxPooling2D object at 0x7f440c030a10>
True <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f440c287c90>
False <keras.layers.core.Activation object at 0x7f440c3a6810>
False <keras.layers.convolutional.ZeroPadding2D object at 0x7f43044d5d50>
False <keras.layers.convolutional.Conv2D object at 0x7f4304ce9ed0>
True <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f440c

In [ ]:
gc.collect()

39487

In [ ]:

# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator

model.fit(
   x=x_train,
   y=y_train,
   batch_size=batch_size,
   epochs=50,
   validation_data=(x_val, y_val),
   callbacks=callbacks,
   steps_per_epoch=train_steps,
   validation_steps=valid_steps,
)

Epoch 1/50
100/100 [==============================] - 17s 129ms/step - loss: 0.9301 - iou_score: 0.0991 - mse: 0.0569 - val_loss: 1.2143 - val_iou_score: 0.0175 - val_mse: 0.1643

Epoch 00001: saving model to model.h5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
100/100 [==============================] - 11s 107ms/step - loss: 0.8873 - iou_score: 0.1403 - mse: 0.0501 - val_loss: 1.0581 - val_iou_score: 0.0253 - val_mse: 0.1159

Epoch 00002: saving model to model.h5
Epoch 3/50
100/100 [==============================] - 11s 107ms/step - loss: 0.8738 - iou_score: 0.1542 - mse: 0.0492 - val_loss: 1.0338 - val_iou_score: 0.0287 - val_mse: 0.1013

Epoch 00003: saving model to model.h5
Epoch 4/50
100/100 [==============================] - 11s 107ms/step - loss: 0.8649 - iou_score: 0.1638 - mse: 0.0498 - val_loss: 1.0182 - val_iou_score: 0.0321 - val_mse: 0.0989

Epoch 00004: saving model to model.h5
Epoch 5/50
100/100 [==============================] - 11s 106ms/step - loss: 0.8563 - iou_score: 0.1735 - mse: 0.0500 - val_loss: 0.9987 - val_iou_score: 0.0420 - val_mse: 0.0837

Epoch 00005: saving model to model.h5
Epoch 6/50
100/100 [==============================] - 11s 107ms/step - loss: 0.8470 - iou_score: 0.1829 - mse: 0.0497 - val_loss

In [ ]:
y_tes = model.predict(X_val[:1000])
y_tes.shape

In [ ]:
ls

model.h5
ref_south_africa_crops_competition_v1_test_labels/
ref_south_africa_crops_competition_v1_test_source_s2_B03/
ref_south_africa_crops_competition_v1_test_source_s2_B04/
ref_south_africa_crops_competition_v1_test_source_s2_B08/
ref_south_africa_crops_competition_v1_test_source_s2_CLM/
ref_south_africa_crops_competition_v1_train_labels/
ref_south_africa_crops_competition_v1_train_source_s2_B03/
ref_south_africa_crops_competition_v1_train_source_s2_B04/
ref_south_africa_crops_competition_v1_train_source_s2_B08/
ref_south_africa_crops_competition_v1_train_source_s2_CLM/
sample_data/


In [ ]:
from google.colab import files
files.download("model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Test Data

In this part we will load the test data (which does not have labels) and predict the crop class for each field

In [8]:
tile_ids_test = competition_test_df['tile_id'].unique()

In [9]:
len(tile_ids_test)

1137

In [11]:
n_obs = 3

In [12]:
X_competition_test = np.empty((0, 3 * n_obs))
X_competition_test  = []
#field_ids_test = np.empty((0, 1))

for i in range (0,len(tile_ids_test)):
    if i%200 == 0:
        print(i)
    tile_id = tile_ids_test[i] 

    tile_df = competition_test_df[competition_test_df['tile_id']==tile_id]
    
    tile_date_times = tile_df[tile_df['satellite_platform']=='s2']['datetime'].unique()
    
    n_X = 0
    for date_time in tile_date_times:
        # Here we retrieve the cloud band, and check if it's cloud free we will load the other bands
        # Otherwise we will pass on to the next observation
        
        clm_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='CLM')]['file_path'].values[0])
        clm_max = np.max(clm_src.read(1))
        
        if clm_max < 25:
            n_X+=1
            b3_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B03')]['file_path'].values[0])
            b3_array = np.expand_dims(b3_src.read(1), axis=2)

            b4_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B04')]['file_path'].values[0])
            b4_array = np.expand_dims(b4_src.read(1), axis=2)

            b8_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B08')]['file_path'].values[0])
            b8_array = np.expand_dims(b8_src.read(1), axis=2)

            X_tile = np.empty((256, 256, 0), dtype=np.uint8)

            X_tile = np.append(X_tile, b3_array, axis = 2)
            X_tile = np.append(X_tile, b4_array, axis = 2)
            X_tile = np.append(X_tile, b8_array, axis = 2)

            X_competition_test.append(X_tile)
        if n_X == n_obs:
            break
        
    #X_competition_test = np.append(X_competition_test, X_tile, axis=0)
X_competition_test = np.array(X_competition_test)
X_competition_test.shape

0
200
400
600
800
1000


(3411, 256, 256, 3)

In [13]:
X_competition_test = np.true_divide(X_competition_test, 255.0)
X_competition_test = np.array(X_competition_test, dtype=np.float16)

In [19]:
X_competition_test.shape

(4548, 256, 256, 3)

In [18]:
gc.collect()

708

In [14]:
field_ids = np.empty((256, 256,0), dtype=np.uint32)
field_ids = []

for i in range (0,len(tile_ids_test)):
    if i%500 == 0:
        print(i)
    tile_id = tile_ids_test[i]  
    
    tile_df = competition_test_df[competition_test_df['tile_id']==tile_id]

    field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
    field_id_array = field_id_src.read(1)
    field_id_a = np.empty((256, 256, 0), dtype=np.uint32)
    field_id_a = np.append(field_id_a, np.expand_dims(field_id_array, axis=2), axis = 2)
    for field_ids_i in range(0, n_obs):
        field_ids.append(field_id_a)
field_ids = np.array(field_ids)
print(np.count_nonzero(~np.isnan(field_ids)))
field_ids.shape

0
500
1000
223543296


(3411, 256, 256, 1)

In [ ]:
field_ids[:10]

In [ ]:
step = 20
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)
y_test = np.empty((0,256,256,10))
for i in range(0,step):
    x_test = preprocess_input(X_competition_test[int(len(X_competition_test)/step)*i : int(len(X_competition_test)/step)*(i+1)])
    print(x_test.shape)
    y_test_step = model.predict(x_test)
    y_test_step = np.array(y_test_step, dtype=np.float16)
    y_test = np.append(y_test, y_test_step, axis = 0)
    gc.collect()
del x_test
del y_test_step
y_test.shape

(170, 256, 256, 3)
(170, 256, 256, 3)
(170, 256, 256, 3)
(170, 256, 256, 3)
(170, 256, 256, 3)
(170, 256, 256, 3)
(170, 256, 256, 3)
(170, 256, 256, 3)
(170, 256, 256, 3)
(170, 256, 256, 3)
(170, 256, 256, 3)
(170, 256, 256, 3)


In [ ]:
y_test = np.around(y_test)
np.unique(y_test, return_counts=True)

(array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13.], dtype=float32),
 array([   19316,  6440924, 50690405, 52987406, 25505828,  9251534,
         2933933,   865567,   248430,    64729,    16382,     3921,
             455,       34]))

In [ ]:
len(y_test.flatten())

149028864

In [27]:
field_ids_flatten = field_ids.flatten()
#del field_ids

In [ ]:
import gc
gc.collect()
data_test = pd.DataFrame()
data_test['field_id'] = field_ids_flatten
del field_ids_flatten

In [87]:
print(type(y_test[0][0][0][0]))
y_test = np.array(y_test, dtype=np.float16)
type(y_test[0][0][0][0])

<class 'numpy.float16'>


numpy.float16

In [89]:
for col_i in range(0,10):
    y_test_flatten = np.array(((y_test.T)[col_i][:][:][:].T).flatten(), dtype = np.float16)
    gc.collect()
    data_test[str(col_i)] = y_test_flatten 
    del y_test_flatten
data_test = data_test[data_test.field_id != 0]
gc.collect()
data_test.head()

,field_id,0,1,2,3,4,5,6,7,8,9
72,102896,0.139893,0.089478,0.036560,0.019974,0.000393,0.009010,0.193115,0.155762,0.352783,0.003067
73,102896,0.109863,0.092163,0.033081,0.018646,0.000340,0.007988,0.189331,0.190186,0.355225,0.002972
74,102896,0.097229,0.093628,0.029053,0.017151,0.000306,0.006954,0.168823,0.229858,0.354248,0.002708
75,102896,0.086975,0.090332,0.023529,0.015854,0.000259,0.005936,0.146851,0.285156,0.342773,0.002439
76,102896,0.079407,0.086060,0.019669,0.014343,0.000236,0.005104,0.123413,0.342285,0.327148,0.002174


In [119]:
pred_df = data_test.groupby('field_id').sum().reset_index()
pred_df.head()

,field_id,0,1,2,3,4,5,6,7,8,9
0,5,7.855469,13.984375,3.919922,1.248047,0.035156,0.400391,18.640625,1341.000000,46.406250,0.336182
1,10,3724.000000,6.199219,361.250000,2.578125,0.227539,4.300781,12.460938,22.875000,11.210938,0.288818
2,11,269.250000,24.109375,1314.000000,4.523438,0.285156,3.255859,180.875000,252.625000,63.718750,0.737793
3,17,771.500000,0.991699,5.781250,609.500000,0.150391,126.625000,3.482422,0.726562,0.254395,1.196289
4,18,122.250000,0.057251,0.787109,0.755859,0.055481,65.625000,0.345215,0.047791,0.043671,0.079407


In [ ]:
y_test_flatten = y_test.flatten()
y_test_flatten = np.where(y_test_flatten < 0, 0, y_test_flatten)
y_test_flatten = np.where(y_test_flatten > 9, 9, y_test_flatten)
np.unique(y_test_flatten, return_counts=True)

(array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32),
 array([   19316,  6440924, 50690405, 52987406, 25505828,  9251534,
         2933933,   865567,   248430,    85521]))

In [ ]:
print(y_test_flatten.shape)
y_test_flatten = np.array(y_test_flatten, dtype = np.uint8)
y_test_flatten.shape

(149028864,)


(149028864,)

In [ ]:
data_test['lables'] = y_test_flatten
del y_test_flatten
data_test = data_test[data_test.field_id != 0]
data_test.head()

,field_id,lables
72,102896,1
73,102896,1
74,102896,1
75,102896,1
76,102896,1


In [ ]:
pred_df = pd.get_dummies(data_test['lables'])
pred_df.head()

,0,1,2,3,4,5,6,7,8,9
72,0,1,0,0,0,0,0,0,0,0
73,0,1,0,0,0,0,0,0,0,0
74,0,1,0,0,0,0,0,0,0,0
75,0,1,0,0,0,0,0,0,0,0
76,0,1,0,0,0,0,0,0,0,0


In [ ]:
pred_df['field_id'] = data_test['field_id']
pred_df.head()

,0,1,2,3,4,5,6,7,8,9,field_id
72,0,1,0,0,0,0,0,0,0,0,102896
73,0,1,0,0,0,0,0,0,0,0,102896
74,0,1,0,0,0,0,0,0,0,0,102896
75,0,1,0,0,0,0,0,0,0,0,102896
76,0,1,0,0,0,0,0,0,0,0,102896


In [ ]:
#pred_df = data_test.groupby('field_id').mean().reset_index()
pred_df = pred_df.groupby('field_id').sum().reset_index()
pred_df.head()

In [120]:
field_id_col = pred_df['field_id']
pred_df = pred_df.drop(columns=['field_id'])
pred_df.head()

,0,1,2,3,4,5,6,7,8,9
0,7.855469,13.984375,3.919922,1.248047,0.035156,0.400391,18.640625,1341.000000,46.406250,0.336182
1,3724.000000,6.199219,361.250000,2.578125,0.227539,4.300781,12.460938,22.875000,11.210938,0.288818
2,269.250000,24.109375,1314.000000,4.523438,0.285156,3.255859,180.875000,252.625000,63.718750,0.737793
3,771.500000,0.991699,5.781250,609.500000,0.150391,126.625000,3.482422,0.726562,0.254395,1.196289
4,122.250000,0.057251,0.787109,0.755859,0.055481,65.625000,0.345215,0.047791,0.043671,0.079407


In [121]:
pred_df = pred_df.div(pred_df.sum(axis=1), axis=0)
pred_df.head()

,0,1,2,3,4,5,6,7,8,9
0,0.005478,0.009750,0.002733,0.000870,0.000024,0.000279,0.013000,0.935059,0.032349,0.000234
1,0.898438,0.001496,0.087158,0.000622,0.000055,0.001038,0.003008,0.005520,0.002705,0.000070
2,0.127319,0.011406,0.621582,0.002140,0.000135,0.001540,0.085571,0.119507,0.030136,0.000349
3,0.507324,0.000652,0.003803,0.400879,0.000099,0.083313,0.002291,0.000478,0.000167,0.000787
4,0.643555,0.000301,0.004143,0.003979,0.000292,0.345459,0.001817,0.000252,0.000230,0.000418


In [123]:
pred_df['field_id'] = field_id_col
pred_df.head()

,0,1,2,3,4,5,6,7,8,9,field_id
0,0.005478,0.009750,0.002733,0.000870,0.000024,0.000279,0.013000,0.935059,0.032349,0.000234,5
1,0.898438,0.001496,0.087158,0.000622,0.000055,0.001038,0.003008,0.005520,0.002705,0.000070,10
2,0.127441,0.011414,0.622070,0.002142,0.000135,0.001541,0.085632,0.119568,0.030151,0.000349,11
3,0.507324,0.000652,0.003803,0.400879,0.000099,0.083313,0.002291,0.000478,0.000167,0.000787,17
4,0.643555,0.000301,0.004143,0.003979,0.000292,0.345459,0.001817,0.000252,0.000230,0.000418,18


In [124]:
pred_df = pred_df.rename(columns={
    0:'No Data',
    1:'Crop_Lucerne/Medics',
    2:'Crop_Planted pastures (perennial)', 
    3:'Crop_Fallow',
    4:'Crop_Wine grapes',
    5:'Crop_Weeds',
    6:'Crop_Small grain grazing',
    7:'Crop_Wheat',
    8:'Crop_Canola',
    9:'Crop_Rooibos'
})
pred_df.head()

,0,1,2,3,4,5,6,7,8,9,field_id
0,0.005478,0.009750,0.002733,0.000870,0.000024,0.000279,0.013000,0.935059,0.032349,0.000234,5
1,0.898438,0.001496,0.087158,0.000622,0.000055,0.001038,0.003008,0.005520,0.002705,0.000070,10
2,0.127441,0.011414,0.622070,0.002142,0.000135,0.001541,0.085632,0.119568,0.030151,0.000349,11
3,0.507324,0.000652,0.003803,0.400879,0.000099,0.083313,0.002291,0.000478,0.000167,0.000787,17
4,0.643555,0.000301,0.004143,0.003979,0.000292,0.345459,0.001817,0.000252,0.000230,0.000418,18


In [125]:
pred_df = pred_df.rename(columns={
    '0':'No Data',
    '1':'Crop_Lucerne/Medics',
    '2':'Crop_Planted pastures (perennial)', 
    '3':'Crop_Fallow',
    '4':'Crop_Wine grapes',
    '5':'Crop_Weeds',
    '6':'Crop_Small grain grazing',
    '7':'Crop_Wheat',
    '8':'Crop_Canola',
    '9':'Crop_Rooibos'
})
pred_df.head()

,No Data,Crop_Lucerne/Medics,Crop_Planted pastures (perennial),Crop_Fallow,Crop_Wine grapes,Crop_Weeds,Crop_Small grain grazing,Crop_Wheat,Crop_Canola,Crop_Rooibos,field_id
0,0.005478,0.009750,0.002733,0.000870,0.000024,0.000279,0.013000,0.935059,0.032349,0.000234,5
1,0.898438,0.001496,0.087158,0.000622,0.000055,0.001038,0.003008,0.005520,0.002705,0.000070,10
2,0.127441,0.011414,0.622070,0.002142,0.000135,0.001541,0.085632,0.119568,0.030151,0.000349,11
3,0.507324,0.000652,0.003803,0.400879,0.000099,0.083313,0.002291,0.000478,0.000167,0.000787,17
4,0.643555,0.000301,0.004143,0.003979,0.000292,0.345459,0.001817,0.000252,0.000230,0.000418,18


In [126]:
sample_sub = pd.read_csv('SampleSubmission.csv')
sample_sub.head()

,Field ID,Crop_Lucerne/Medics,Crop_Planted pastures (perennial),Crop_Fallow,Crop_Wine grapes,Crop_Weeds,Crop_Small grain grazing,Crop_Wheat,Crop_Canola,Crop_Rooibos
0,5,0,0,0,0,0,0,0,0,0
1,10,0,0,0,0,0,0,0,0,0
2,11,0,0,0,0,0,0,0,0,0
3,17,0,0,0,0,0,0,0,0,0
4,18,0,0,0,0,0,0,0,0,0


In [127]:
for column in pred_df.columns[:-1]:
    sample_sub[column] = pred_df[column]
sample_sub.head()

,Field ID,Crop_Lucerne/Medics,Crop_Planted pastures (perennial),Crop_Fallow,Crop_Wine grapes,Crop_Weeds,Crop_Small grain grazing,Crop_Wheat,Crop_Canola,Crop_Rooibos,No Data
0,5,0.009750,0.002733,0.000870,0.000024,0.000279,0.013000,0.935059,0.032349,0.000234,0.005478
1,10,0.001496,0.087158,0.000622,0.000055,0.001038,0.003008,0.005520,0.002705,0.000070,0.898438
2,11,0.011414,0.622070,0.002142,0.000135,0.001541,0.085632,0.119568,0.030151,0.000349,0.127441
3,17,0.000652,0.003803,0.400879,0.000099,0.083313,0.002291,0.000478,0.000167,0.000787,0.507324
4,18,0.000301,0.004143,0.003979,0.000292,0.345459,0.001817,0.000252,0.000230,0.000418,0.643555


In [128]:
sample_sub.describe()

,Field ID,Crop_Lucerne/Medics,Crop_Planted pastures (perennial),Crop_Fallow,Crop_Wine grapes,Crop_Weeds,Crop_Small grain grazing,Crop_Wheat,Crop_Canola,Crop_Rooibos,No Data
count,35295.000000,35294.000000,3.529400e+04,35294.000000,35294.000000,3.529400e+04,35294.000000,35294.000000,35294.000000,35294.000000,35294.000000
mean,61139.709052,0.043671,1.206055e-01,0.052917,0.207397,5.581665e-02,0.065491,0.093262,0.012611,0.030487,0.317871
std,35462.690869,0.137939,2.322998e-01,0.174683,0.369385,1.761475e-01,0.166870,0.233276,0.023270,0.147827,0.355713
min,5.000000,0.000000,5.960464e-08,0.000000,0.000000,2.384186e-07,0.000000,0.000000,0.000000,0.000000,0.000074
25%,30433.500000,0.000238,2.149582e-03,0.000502,0.000044,4.878044e-04,0.000575,0.000138,0.000060,0.000098,0.022675
50%,61097.000000,0.001719,7.904053e-03,0.001754,0.000251,1.499176e-03,0.003788,0.000947,0.000953,0.000227,0.134644
75%,91862.500000,0.008789,9.582520e-02,0.005142,0.197266,5.470276e-03,0.024399,0.014618,0.014870,0.000597,0.580078
max,122734.000000,0.993652,9.990234e-01,0.999023,0.998535,9.995117e-01,0.984863,0.998047,0.213745,0.999023,1.000000


In [130]:
del_sub = (sample_sub['Field ID'] - pred_df['field_id'])
print(sum(list(del_sub[:])))
del del_sub

0.0


In [104]:
sample_sub = sample_sub.drop(['No Data'], axis=1)
sample_sub.head()

,Field ID,Crop_Lucerne/Medics,Crop_Planted pastures (perennial),Crop_Fallow,Crop_Wine grapes,Crop_Weeds,Crop_Small grain grazing,Crop_Wheat,Crop_Canola,Crop_Rooibos
0,5,0.018692,0.004436,0.001561,0.000022,0.000656,0.404053,0.293701,0.267334,0.000163
1,10,0.000768,0.729492,0.000425,0.000042,0.000750,0.001585,0.000461,0.002733,0.000101
2,11,0.001736,0.114502,0.001540,0.000028,0.001093,0.277100,0.003944,0.052002,0.000090
3,17,0.000272,0.004692,0.516113,0.000053,0.380859,0.003382,0.003397,0.003014,0.004509
4,18,0.000259,0.002058,0.167725,0.000027,0.023743,0.001114,0.001004,0.004910,0.001620


In [132]:
# Write the predicted probabilites to a csv for submission
sample_sub.to_csv('baseline_submission4.csv', index=False)

In [133]:
from google.colab import files
files.download("baseline_submission4.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
ls

baseline_submission3.csv
baseline_submission4.csv
ref_south_africa_crops_competition_v1_test_labels/
ref_south_africa_crops_competition_v1_test_source_s2_B03/
ref_south_africa_crops_competition_v1_test_source_s2_B04/
ref_south_africa_crops_competition_v1_test_source_s2_B08/
ref_south_africa_crops_competition_v1_test_source_s2_CLM/
ref_south_africa_crops_competition_v1_train_labels/
ref_south_africa_crops_competition_v1_train_source_s2_B03/
ref_south_africa_crops_competition_v1_train_source_s2_B04/
ref_south_africa_crops_competition_v1_train_source_s2_B08/
ref_south_africa_crops_competition_v1_train_source_s2_CLM/
sample_data/
SampleSubmission.csv
